In [1]:
from lib.utilities import getInverseAction

In [2]:
#Env Init:
from lib.TaxiEnv import TaxiEnv
import numpy as np
env = TaxiEnv()
env.print = False

#Action hopper - Hopper1 is "current itteration", Hopper2 is "nextStep" 
actionHopper = []
actionHopper2 = []

#Just a basic "Been here before" paths pruning
states=[]

#First manual Itteration - needs to make main cycle more generic
startState = env.getLastState()
actionMask = env.getLastActionMask()
#print(actionMask)
#Whole path, states, costs and also actions
returnVal = []

akceId = 0
for i in actionMask:
            #If action is possible:
            if(i == 1):
                actionHopper.append((startState,akceId,0))
                states.append(startState)
                if(akceId >= 4):
                    returnVal.append([startState,akceId,0,[]])
            akceId += 1
col, row = env.getPos(startState)
env.printCurState()
# print(actionHopper)

#Values used for return - "we got to the result" flag
dontReturn = True
#Basic heuristics - make sure we didn't ran into an infinite cycle
counter = 0

#Have we already solved it in first itter?
if(len(returnVal) == 0):
    #While result doesn't exist and we still got some actions to do:
    while (dontReturn and len(actionHopper) > 0):
        #Check if we are not Too deep
        if(counter > 15):
            break
        counter += 1
        #For all actions in list:
        for i in actionHopper:
                
                #Set the state from action
                env.setState(i[0])
                #Make a move
                (next_state, reward, done, info1, info2) = env.move(i[1])
                #If we have already visited this state - go for next action
                if(next_state in states):
                    continue

                #Check if we didn't score target
                if(reward == 20):
                    #If yes - return
                    dontReturn = False
                    returnVal = i
                    break

                #Get possible actions of current state
                actionMask = info2["action_mask"]
                akceId = 0
                #For all possible actions:
                for j in actionMask:
                    #If the action is possible
                    if(j == 1):
                        #If action is not reverse of the current one:
                        #(we really don't want to go back - efficiency)
                        if(akceId != getInverseAction(i[1])):
                            #Append that action to the hopper
                            actionHopper2.append([next_state,akceId,i[2]+1,i])
                            #If the action is final - we got a target so return it
                            if(akceId >= 4):
                                dontReturn = False
                                returnVal = [next_state,akceId,i[2]+1,i]
                                break
                    akceId += 1
                #Mark this state as completed
                states.append(i[0])
                #Check if we should return something
                if(not dontReturn):
                    break
        actionHopper = actionHopper2
        actionHopper2 = []

#Indexy akcí: 0 - down, 1 - up, 2 - right, 3 - left, 4 - pick, 5 - put. 
print(returnVal)

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+


[2, 4, 6, [22, 3, 5, [122, 1, 4, [222, 1, 3, [242, 3, 2, [262, 3, 1, (282, 3, 0)]]]]]]
